In [89]:
from langchain_community.document_loaders import UnstructuredFileLoader
import glob

In [90]:
documents=[]
folder_path = "Data/*"

documents = []

# Loop through all files
for file in glob.glob(folder_path):
    loader = UnstructuredFileLoader(file)
    docs = loader.load()
    documents.extend(docs)


In [91]:
print(len(documents[1].page_content))

20927


In [92]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [93]:
chunks=splitter.split_documents(documents)

In [94]:
print(chunks[0].page_content)

JournalofBusinessResearch177(2024)114655

Availableonline9April20240148-2963/©2024TheAuthor(s).PublishedbyElsevierInc.ThisisanopenaccessarticleundertheCCBYlicense(http://creativecommons.org/licenses/by/4.0/).

Contents lists available at ScienceDirect

Journal of Business Research

journal homepage: www.elsevier.com/locate/jbusres

Unstructured data research in business: Toward a structured approach


In [95]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer("BAAI/bge-base-en-v1.5")

In [106]:
document_string=[c.page_content for c in chunks]
embedding=model.encode(document_string)
len(embedding),len(document_string)

(296, 296)

In [107]:
len(embedding[0])

768

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.environ["PINECONE_API_KEY"]
from pinecone import Pinecone, ServerlessSpec
pc=Pinecone(api_key=api_key)


In [112]:
indexName="doc-index"
dimension=768
metric="cosine"

In [ ]:
pc.create_index(name=indexName,dimension=dimension ,metric=metric,spec=ServerlessSpec(cloud="aws",region="us-east-1"))

In [114]:
Index=pc.Index(indexName)

In [115]:
vectors=[]
for i in range(0,len(embedding)):
    obj={
        "id":str(i+1),
        "values":embedding[i],
        "metadata":{
        "text":chunks[i].page_content
        }
    }
    vectors.append(obj)

In [116]:
Index.upsert(vectors)

{'upserted_count': 296}

In [143]:
Query="What is TF-IDF,explain it with an easy example"
emb_query=model.encode(Query,convert_to_tensor=False).tolist()
response=Index.query(vector=emb_query,top_k=5,include_metadata=True,include_values=True)

In [144]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

HF_API_KEY=os.environ["HUGGING_FACE_API_KEY"]
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x22B-Instruct-v0.1",
    huggingfacehub_api_token=HF_API_KEY,          
    task="text-generation",
    temperature=0,
    max_new_tokens=1000
)
chat_model=ChatHuggingFace(llm=llm)

In [160]:
context=response.matches

prompt = f"""
You are User Query Chatbot , that can reluctantly and accurately answer 
user Question  based on Context 
!!!IF detail explanation is required then give it !!!
Customer Data:
{context}

User Question:
{Query}

Answer in simple and clear English.
consider best match 

Important : Give Only Answer no other text
"""
print(chat_model.invoke(prompt).content)

 TF-IDF stands for Term Frequency-Inverse Document Frequency. It's a numerical statistic used in information retrieval and text mining that reflects the importance of a word to a document in a collection or corpus. TF-IDF is calculated as the product of two statistics: the term frequency (TF) and the inverse document frequency (IDF). The term frequency measures how often a term appears in a document, while the inverse document frequency measures how common or rare the term is across the entire collection of documents. An easy example can be: Consider two sentences - "The sun rises in the east" and "The sun sets in the west". If we want to find out how important the word "rises" is in the first sentence, we calculate TF as 1 ("rises" appears once in the first sentence) and IDF as log(2/1) = 0.301 (as "rises" appears only once across two sentences). So, TF-IDF for "rises" is 1*0.301 = 0.301, reflecting its importance in the first sentence.
